In [1]:
%matplotlib inline

In [2]:
import time
import copy

import numpy as np
import pandas as pd

import torch
from torch.utils.data import DataLoader
import torchvision

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

In [4]:
transforms = torchvision.transforms.Compose([
#     torchvision.transforms.transforms.ToPILImage(),
    torchvision.transforms.RandomResizedCrop(224),
    torchvision.transforms.RandomRotation(180),
    torchvision.transforms.RandomHorizontalFlip(p=0.5),
    torchvision.transforms.RandomVerticalFlip(p=0.5),
    torchvision.transforms.RandomGrayscale(p=0.1),
    torchvision.transforms.ToTensor()
#     torchvision.transforms.TenCrop(224), # this is a list of PIL Images
#     torchvision.transforms.Lambda(lambda crops: torch.stack([torchvision.transforms.ToTensor()(crop) for crop in crops])) # returns a 4D tensor
])

In [5]:
train_dataset = torchvision.datasets.ImageFolder("data/drive-download-20190413T124209Z-001/", transform=transforms)

In [6]:
model = torchvision.models.resnet50(pretrained='imagenet')

In [7]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [8]:
model = torch.nn.Sequential(
    model,
    torch.nn.Linear(1000, 5),
    torch.nn.Softmax()
)

In [9]:
EPOCHS = 10
BATCH_SIZE = 32

In [10]:
dataloaders = {
    "train": DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True),
    "val": DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=False)
}

In [11]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    model = model.to(device)
    
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
#                 bs, ncrops, c, h, w = inputs.size()

                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
#                         outputs = model(inputs.view(-1, c, h, w)) # fuse batch size and ncrops
#                         outputs_avg = outputs.view(bs, ncrops, -1).mean(1) # avg over crops
                
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [12]:
criterion = torch.nn.CrossEntropyLoss()

In [13]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.003)

In [14]:
model, val_acc_history = train_model(model, dataloaders=dataloaders, criterion=criterion, optimizer=optimizer, num_epochs=20)

Epoch 0/19
----------


c:\users\gx\appdata\local\programs\python\python36\lib\site-packages\torch\nn\modules\container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


KeyboardInterrupt: 

In [17]:
torch.save(model, "model.pth")

In [15]:
train_dataset.class_to_idx

{'aluminium tin': 0, 'cardboard': 1, 'glass': 2, 'paper': 3, 'plastic': 4}